In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from azure.storage.blob import BlockBlobService

### Adatbetoltes

In [7]:
df = pd.read_csv('DataSet_LakasP.csv', sep=',')

In [8]:
df.head(2)

,county,city,postcode,property_type,property_subtype,property_condition_type,property_floor,building_floor_count,view_type,orientation,...,elevator_type,room_cnt,small_room_cnt,created_at,property_area,balcony_area,price_created_at,ad_view_cnt,active_days,nr
0,Budapest,Budapest XII.,NaN,flat,brick flat (for sale),good,1,NaN,street view,NaN,...,yes,2.0,1.0,2015-02-09,65.0,0.0,23.5,605.0,119.0,4
1,Budapest,Budapest I.,1016.0,flat,brick flat (for sale),novel,2,NaN,street view,NaN,...,yes,1.0,1.0,2015-02-09,45.0,0.0,20.0,49.0,25.0,12


In [9]:
df.describe()

,postcode,room_cnt,small_room_cnt,property_area,balcony_area,price_created_at,ad_view_cnt,active_days,nr
count,49587.000000,78542.000000,78542.000000,78542.000000,78542.000000,78542.000000,78542.000000,78542.000000,78542.000000
mean,1103.360780,1.467673,0.559942,48.440771,1.953108,19.340736,259.597273,44.174060,196333.134158
std,50.769527,0.599837,0.737123,12.716479,4.677153,8.900928,512.343391,47.822375,113303.860736
min,1011.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,1.000000,4.000000
25%,1064.000000,1.000000,0.000000,40.000000,0.000000,13.200000,42.000000,11.000000,97960.750000
50%,1101.000000,1.000000,0.000000,50.000000,0.000000,16.900000,103.000000,28.000000,196091.500000
75%,1142.000000,2.000000,1.000000,60.000000,3.000000,23.900000,263.000000,61.000000,294507.500000
max,1239.000000,7.000000,4.000000,70.000000,97.000000,99.600000,28096.000000,537.000000,394181.000000


In [11]:
df.dtypes

county                      object
city                        object
postcode                   float64
property_type               object
property_subtype            object
property_condition_type     object
property_floor              object
building_floor_count        object
view_type                   object
orientation                 object
garden_access               object
heating_type                object
elevator_type               object
room_cnt                   float64
small_room_cnt             float64
created_at                  object
property_area              float64
balcony_area               float64
price_created_at           float64
ad_view_cnt                float64
active_days                float64
nr                           int64
dtype: object

In [10]:
df.shape

(78542, 22)

### Modell epites

In [ ]:
df.columns

In [15]:
input_cols = ['room_cnt','property_area','balcony_area']
X = df[input_cols]
y = df['price_created_at']

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2)

In [18]:
model = RandomForestRegressor(random_state=666, n_estimators=300)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
           oob_score=False, random_state=666, verbose=0, warm_start=False)

### Modell mentes azure-ba

In [19]:
block_blob_service = BlockBlobService(account_name='...',
                                      account_key='...')

In [20]:
from io import BytesIO
import joblib

In [21]:
joblib.dump(value=model, filename='model.pkl')

['model.pkl']

In [22]:
block_blob_service.create_blob_from_path(container_name='', blob_name='', file_path='')

In [28]:
model2 = block_blob_service.get_blob_to_bytes('bence', 'model.pkl')

In [24]:
model2 = joblib.load(BytesIO(model2.content))

In [27]:
model2.predict([[3, 70, 10]])

array([36.4304582])